In [1]:
# Version check. Written for Python 3.5.4
import sys
print(sys.version)
#3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
#[GCC 7.2.0]

import matplotlib
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.axes as axes
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from sympy import *
from sympy.utilities.autowrap import autowrap
import re
import numpy.fft as fft
from scipy.signal import argrelextrema
import random
import warnings
from matplotlib.lines import Line2D

from os import listdir
from os.path import isfile, join

%matplotlib inline

3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]


In [9]:
# Define the path to the input folder containing the output parameters
inputFolder = "/Users/muriel/Documents/LACDR/Projects/E2S/Modeling/Output/E2Model/version2/MN10/E2/20221230_165912/"#M010/E2/20220613_171703/" # Write down the path to the folder that
inputFolderAllSets = "/Users/muriel/Documents/LACDR/Projects/E2S/Modeling/Output/E2Model/version2/MN10/E2/AllSets/"#M010/E2/20220613_171703/" # Write down the path to the folder that
# contains the parameter estimates of all the finished runs

# Define the path to the parameter estimates
filename = "20221230_165912_MH_MN10Model_parameterEstimates_parmset1_cost_294.44.csv"

# Read in the file with the parameter estimates
file = inputFolder+filename

In [3]:
# Read in the file with the parameter estimates
file = inputFolder+filename

# Define the parameters
parameterEst = pd.read_csv(file)

pAllNew = parameterEst["est_value"].values

# Make the parameters global variables
for i in parameterEst.iterrows():
    sText = str(parameterEst.iloc[i[0], 0]) + " = " + str(parameterEst.iloc[i[0], 2])
    print(sText)
    exec(sText)

CONC2_init = 0.042460989840139934
CONC3_init = 0.1659808914688818
CONC4_init = 0.2366562074735528
CONC5_init = 0.3743171777243467
CONC6_init = 0.8372565041030933
ER_init = 0.12604737829819276
PR_init = 0.7941287753443627
GREB1_init = 0.9134348990318942
TFF1_init = 0.9671899408866416
d_er = 0.02227131880644441
d_pr = 0.06562196867299717
d_greb1 = 0.3725773184940744
d_tff1 = 0.0
b_e2er = 0.9999999999
d_e2er = 0.24597683777854515
b_e2erpr = 0.025412532897680137
d_e2erpr = 0.8141993330728129
b_e2erprgreb1 = 0.9999999527096904
d_e2erprgreb1 = 0.004810306600735929
stim_pr = 67.69653835443742
stim_greb1 = 999.9999981621162
stim_tff1 = 10.469512336084826
s_er = 0.002807241346795553
s_pr = 0.05211229361797338
s_greb1 = 0.34032512530020875
s_tff1 = 0.0


In [4]:
CONC1_init = 0.001
Ce2er_init = 0
Ce2erpr_init = 0
Ce2erprgreb1_init = 0

In [5]:
# PART 1: constant oscillations
def CellCycle_default(z,t):    
    
    # State vars
    E2, ER, Ce2er, PR, Ce2erpr, GREB1,  Ce2erprgreb1, TFF1  = z
    
    # E2 signaling
    dE2 = - b_e2er * E2 * ER / (1 + E2 + ER)
    dER = s_er - b_e2er * E2 * ER / (1 + E2 + ER) - d_er * ER 
    dCe2er = b_e2er * E2 * ER / (1 + E2 + ER) - b_e2erpr * Ce2er * PR / (1 + Ce2er + PR) - d_e2er * Ce2er

    # Formation of second complex
    dPR = s_pr + stim_pr * Ce2erprgreb1 / (1 + Ce2erprgreb1) - b_e2erpr * Ce2er * PR / (1 + Ce2er + PR) - d_pr * PR 
    dCe2erpr = b_e2erpr * Ce2er * PR / (1 + Ce2er + PR) - b_e2erprgreb1 * Ce2erpr * GREB1 / (1 + Ce2erpr + GREB1) - d_e2erpr * Ce2erpr

    # Formation of third complex
    dGREB1 = s_greb1 - b_e2erprgreb1 * Ce2erpr * GREB1 / (1 + Ce2erpr + GREB1) + stim_greb1 * Ce2erprgreb1 / (1 + Ce2erprgreb1) - d_greb1 * GREB1
    dCe2erprgreb1 = b_e2erprgreb1 * Ce2erpr * GREB1 / (1 + Ce2erpr + GREB1) - d_e2erprgreb1 * Ce2erprgreb1
    dTFF1 = s_tff1 + stim_tff1 * Ce2er * PR  / (1 + Ce2er + PR) - d_tff1 * TFF1

    return [dE2, dER, dCe2er, dPR, dCe2erpr, dGREB1, dCe2erprgreb1, dTFF1]



In [6]:
# Recalculate steady state constraints
s_er = d_er * ER_init
s_pr = d_pr * PR_init 
s_greb1 = d_greb1 * GREB1_init
s_tff1 = d_tff1 * TFF1_init

s_er, s_pr, s_greb1, s_tff1

(0.0028072413467955535, 0.05211229361797338, 0.34032512530020875, 0.0)

In [7]:
# Check steady states

# E2 signaling
dE2 = - b_e2er * 0 * ER_init / (1 + 0 + ER_init)
dER = s_er - b_e2er * 0 * ER_init / (1 + 0 + ER_init) - d_er * ER_init
dCe2er = b_e2er * 0 * ER_init / (1 + 0 + ER_init) - b_e2erpr * Ce2er_init * PR_init / (1 + Ce2er_init + PR_init) - d_e2er * Ce2er_init

# Formation of second complex
dPR = s_pr + stim_pr * Ce2erprgreb1_init / (1 + Ce2erprgreb1_init) - b_e2erpr * Ce2er_init * PR_init / (1 + Ce2er_init + PR_init) - d_pr * PR_init
dCe2erpr = b_e2erpr * Ce2er_init * PR_init / (1 + Ce2er_init + PR_init) - b_e2erprgreb1 * Ce2erpr_init * GREB1_init / (1 + Ce2erpr_init + GREB1_init) - d_e2erpr * Ce2erpr_init

# Formation of third complex
dGREB1 = s_greb1 - b_e2erprgreb1 * Ce2erpr_init * GREB1_init / (1 + Ce2erpr_init + GREB1_init) + stim_greb1 * Ce2erprgreb1_init / (1 + Ce2erprgreb1_init) - d_greb1 * GREB1_init
dCe2erprgreb1 = b_e2erprgreb1 * Ce2erpr_init * GREB1_init / (1 + Ce2erpr_init + GREB1_init) - d_e2erprgreb1 * Ce2erprgreb1_init
dTFF1 = s_tff1 + stim_tff1 * Ce2er_init * PR_init  / (1 + Ce2er_init + PR_init) - d_tff1 * TFF1_init

# print outcome
dE2, dER, dCe2er, dPR, dCe2erpr, dGREB1, dCe2erprgreb1, dTFF1



(-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [8]:
# PART 0: Run simulation and save output for plotting in R. 

# Initial conditions
z_part0 = [CONC1_init, ER_init, Ce2er_init, PR_init, Ce2erpr_init, GREB1_init, Ce2erprgreb1_init, TFF1_init]

# PART 1 time span 
t_part0 = np.linspace(1, 55, 55)

# Run simulation
out_part0 = odeint(CellCycle_default, z_part0, t_part0)

d = {'dose_nM': np.repeat(0.001,55*3), 'timeID': np.hstack(np.repeat([np.arange(1,56,1)],3, axis = 0)), "StateVar": ["GREB1"]*55 + ["PR"]*55 + ["TFF1"]*55,
    "GFP": np.hstack((out_part0[:,5]+out_part0[:,6],out_part0[:,3]+out_part0[:,4]+out_part0[:,6],out_part0[:,7]))}

simu_data = pd.DataFrame(data = d)
simulations = [out_part0[:,2::]]

for dose,conc in enumerate([CONC2_init, CONC3_init, CONC4_init, CONC5_init, CONC6_init]):

    # Initial conditions
    z_part0 = [conc, ER_init, Ce2er_init, PR_init, Ce2erpr_init, GREB1_init, Ce2erprgreb1_init, TFF1_init]

    # PART 1 time span 
    t_part0 = np.linspace(1, 55, 55)

    # Run simulation
    out_part0 = odeint(CellCycle_default, z_part0, t_part0)
    
    nominal_conc = [0.01,0.1,1,10,100][dose]

    d = {'dose_nM': np.repeat(nominal_conc,55*3), 'timeID': np.hstack(np.repeat([np.arange(1,56,1)],3, axis = 0)), "StateVar": ["GREB1"]*55 + ["PR"]*55 + ["TFF1"]*55,
        "GFP": np.hstack((out_part0[:,5]+out_part0[:,6],out_part0[:,3]+out_part0[:,4]+out_part0[:,6],out_part0[:,7]))}
    
    data = pd.DataFrame(data = d)
    
    simu_data = pd.concat([simu_data, data])

    simulations.append(out_part0[:,2::])

simu_data

#simu_data.to_csv("MN10_simu_data.csv")




,dose_nM,timeID,StateVar,GFP
0,0.001,1,GREB1,0.913435
1,0.001,2,GREB1,0.913452
2,0.001,3,GREB1,0.913624
3,0.001,4,GREB1,0.914118
4,0.001,5,GREB1,0.915011
...,...,...,...,...
160,100.000,51,TFF1,6.721506
161,100.000,52,TFF1,6.808944
162,100.000,53,TFF1,6.896556
163,100.000,54,TFF1,6.984335


In [22]:
for idx,f in enumerate(listdir(inputFolderAllSets)):
    if "parameterEstimates" in f:
        file = inputFolderAllSets+f
        
        # Get parmset index
        #idx = f[48:57]
        # Remove underscore from idx
        #idx = idx.replace("_","")

        # Get parmset cost
        cost = f[57:69]
        # Remove underscore from cost
        cost = cost.replace("_","")
        # Remove trailing dot from cost
        cost = cost.strip('.')
        print(cost)

cost294.44
cost287.67
cost294.44
cost294.44
cost294.44
cost326.03
cost294.44
cost294.44
cost294.44
cost287.67
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost287.67
cost294.44
cost294.44
cost369.00
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost287.67
cost294.44
cost294.44
cost297.17
cost294.44
cost287.67
cost294.44
cost326.03
cost287.67
cost294.44
cost287.67
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost287.67
cost307.30
cost307.30
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44
cost294.44


In [24]:
for idx,f in enumerate(listdir(inputFolderAllSets)):
    print(f)
    if "parameterEstimates" in f:
        file = inputFolderAllSets+f
        
        # Get parmset index
        #idx = f[48:57]
        # Remove underscore from idx
        #idx = idx.replace("_","")

        # Get parmset cost
        cost = f[57:69]
        # Remove underscore from cost
        cost = cost.replace("_","")
        # Remove trailing dot from cost
        cost = cost.strip('.')

        # Define the parameters
        parameterEst = pd.read_csv(file)

        pAllNew = parameterEst["est_value"].values

        # Make the parameters global variables
        for i in parameterEst.iterrows():
            sText = str(parameterEst.iloc[i[0], 0]) + " = " + str(parameterEst.iloc[i[0], 2])
            print(sText)
            exec(sText)
            
            # Recalculate steady state constraints
            s_er = d_er * ER_init
            s_pr = d_pr * PR_init 
            s_greb1 = d_greb1 * GREB1_init
            s_tff1 = d_tff1 * TFF1_init

        # PART 0: Run simulation and save output for plotting in R. 

        # Initial conditions
        z_part0 = [CONC1_init, ER_init, Ce2er_init, PR_init, Ce2erpr_init, GREB1_init, Ce2erprgreb1_init, TFF1_init]

        # PART 1 time span 
        t_part0 = np.linspace(1, 55, 55)

        # Run simulation
        out_part0 = odeint(CellCycle_default, z_part0, t_part0)

        d = {'dose_nM': np.repeat(0.001,55*3), 'timeID': np.hstack(np.repeat([np.arange(1,56,1)],3, axis = 0)), "StateVar": ["GREB1"]*55 + ["PR"]*55 + ["TFF1"]*55,
            "GFP": np.hstack((out_part0[:,5]+out_part0[:,6],out_part0[:,3]+out_part0[:,4]+out_part0[:,6],out_part0[:,7]))}

        simu_data = pd.DataFrame(data = d)
        simulations = [out_part0[:,2::]]

        for dose,conc in enumerate([CONC2_init, CONC3_init, CONC4_init, CONC5_init, CONC6_init]):

            # Initial conditions
            z_part0 = [conc, ER_init, Ce2er_init, PR_init, Ce2erpr_init, GREB1_init, Ce2erprgreb1_init, TFF1_init]

            # PART 1 time span 
            t_part0 = np.linspace(1, 55, 55)

            # Run simulation
            out_part0 = odeint(CellCycle_default, z_part0, t_part0)

            nominal_conc = [0.01,0.1,1,10,100][dose]

            d = {'dose_nM': np.repeat(nominal_conc,55*3), 'timeID': np.hstack(np.repeat([np.arange(1,56,1)],3, axis = 0)), "StateVar": ["GREB1"]*55 + ["PR"]*55 + ["TFF1"]*55,
                "GFP": np.hstack((out_part0[:,5]+out_part0[:,6],out_part0[:,3]+out_part0[:,4]+out_part0[:,6],out_part0[:,7]))}

            data = pd.DataFrame(data = d)

            simu_data = pd.concat([simu_data, data])

            simulations.append(out_part0[:,2::])

        simu_data

        simu_data.to_csv("MN10_simu_data_parmset" + str(idx+1) + "_" + cost + ".csv")
        

20221230_165912_MH_MN10Model_parameterEstimates_parmset10_cost_294.44.csv
CONC2_init = 0.042452361792327926
CONC3_init = 0.16594272719756625
CONC4_init = 0.2366068932302563
CONC5_init = 0.3742373879360392
CONC6_init = 0.8369244372401308
ER_init = 0.12599067881600884
PR_init = 0.7941259649443617
GREB1_init = 0.9134314659686892
TFF1_init = 0.9671872737388963
d_er = 0.022275294136385102
d_pr = 0.06560091943258807
d_greb1 = 0.3723335980216152
d_tff1 = 0.0
b_e2er = 0.9999999999980111
d_e2er = 0.2460347630682296
b_e2erpr = 0.02544330469057302
d_e2erpr = 0.8157390116623854
b_e2erprgreb1 = 0.9999999999995752
d_e2erprgreb1 = 0.0048204478337721185
stim_pr = 67.72405429641303
stim_greb1 = 999.9999997386516
stim_tff1 = 10.476384150824867
s_er = 0.0028064794290694212
s_pr = 0.05209539344564134
s_greb1 = 0.3401012242702806
s_tff1 = 0.0
20221230_165912_MH_MN10Model_parameterEstimates_parmset9_cost_287.67.csv
CONC2_init = 0.12545787488656535
CONC3_init = 0.5610447313583248
CONC4_init = 0.9232560020402

20221230_165912_MH_MN10Model_parameterEstimates_parmset7_cost_294.44.csv
CONC2_init = 0.04244350491381507
CONC3_init = 0.16590602320380068
CONC4_init = 0.23654987957241036
CONC5_init = 0.3741125329787273
CONC6_init = 0.8363742011127389
ER_init = 0.12597015836930409
PR_init = 0.7941242280935967
GREB1_init = 0.9134244562130874
TFF1_init = 0.967187283917899
d_er = 0.022272551715232982
d_pr = 0.06561828929591278
d_greb1 = 0.37241541023078345
d_tff1 = 0.0
b_e2er = 0.9999999999960412
d_e2er = 0.24604248506957715
b_e2erpr = 0.025459011449998598
d_e2erpr = 0.8160299775167082
b_e2erprgreb1 = 0.9999999999857112
d_e2erprgreb1 = 0.00481624693556709
stim_pr = 67.72017174142394
stim_greb1 = 999.9999674539661
stim_tff1 = 10.479619779954627
s_er = 0.002805676866856413
s_pr = 0.05210907333593905
s_greb1 = 0.3401733435754273
s_tff1 = 0.0
20240522_092042_MH_MN10Model_parameterEstimates_parmset10_cost_294.44.csv
CONC2_init = 0.04244921630350618
CONC3_init = 0.16592749258480535
CONC4_init = 0.2365836807855

20221230_165912_MH_MN10Model_parameterEstimates_parmset1_cost_294.44.csv
CONC2_init = 0.042460989840139934
CONC3_init = 0.1659808914688818
CONC4_init = 0.2366562074735528
CONC5_init = 0.3743171777243467
CONC6_init = 0.8372565041030933
ER_init = 0.12604737829819276
PR_init = 0.7941287753443627
GREB1_init = 0.9134348990318942
TFF1_init = 0.9671899408866416
d_er = 0.02227131880644441
d_pr = 0.06562196867299717
d_greb1 = 0.3725773184940744
d_tff1 = 0.0
b_e2er = 0.9999999999
d_e2er = 0.24597683777854515
b_e2erpr = 0.025412532897680137
d_e2erpr = 0.8141993330728129
b_e2erprgreb1 = 0.9999999527096904
d_e2erprgreb1 = 0.004810306600735929
stim_pr = 67.69653835443742
stim_greb1 = 999.9999981621162
stim_tff1 = 10.469512336084826
s_er = 0.002807241346795553
s_pr = 0.05211229361797338
s_greb1 = 0.34032512530020875
s_tff1 = 0.0
20240522_092042_MH_MN10Model_parameterEstimates_parmset3_cost_294.44.csv
CONC2_init = 0.04246197431706169
CONC3_init = 0.1659801373121517
CONC4_init = 0.2366561291141668
CONC

20240522_092042_MH_MN10Model_parameterEstimates_parmset9_cost_294.44.csv
CONC2_init = 0.04244564590934382
CONC3_init = 0.16591642431244366
CONC4_init = 0.2365619344888089
CONC5_init = 0.3741086390055429
CONC6_init = 0.8363663659346108
ER_init = 0.12597260284781506
PR_init = 0.7941241728942849
GREB1_init = 0.9134244743035852
TFF1_init = 0.9671843570598088
d_er = 0.02227906426586276
d_pr = 0.06561243674398116
d_greb1 = 0.3724042455228776
d_tff1 = 1.0334856446318382e-163
b_e2er = 0.9999999999821072
d_e2er = 0.2461070541027346
b_e2erpr = 0.025449367658375785
d_e2erpr = 0.8152727151057082
b_e2erprgreb1 = 0.9999998515294394
d_e2erprgreb1 = 0.004816407416021657
stim_pr = 67.71924150347675
stim_greb1 = 999.9999012493704
stim_tff1 = 10.480765187723561
s_er = 0.002806551714584477
s_pr = 0.052104422060892636
s_greb1 = 0.3401631521951577
s_tff1 = 9.995711487337865e-164
20240522_092939_MH_MN10Model_parameterEstimates_parmset14_cost_294.44.csv
CONC2_init = 0.04244311780762629
CONC3_init = 0.16590233

20240522_092042_MH_MN10Model_parameterEstimates_parmset8_cost_287.67.csv
CONC2_init = 0.1255103570784934
CONC3_init = 0.5612758501449443
CONC4_init = 0.9235639460839224
CONC5_init = 2.025656468323472
CONC6_init = 999.9820372076961
ER_init = 0.14953757635481055
PR_init = 0.7867282868536121
GREB1_init = 0.9221323470160809
TFF1_init = 0.9700739109364972
d_er = 0.07669742721175621
d_pr = 0.0
d_greb1 = 0.05654244452048478
d_tff1 = 0.03467760200327048
b_e2er = 0.4416002568411164
d_e2er = 0.2038863080533168
b_e2erpr = 0.049022315006567264
d_e2erpr = 0.6910160634725611
b_e2erprgreb1 = 0.330824383900437
d_e2erprgreb1 = 0.5282273837922834
stim_pr = 125.87734365927912
stim_greb1 = 999.9999993092244
stim_tff1 = 7.494721507127641
s_er = 0.01146914737789552
s_pr = 0.0
s_greb1 = 0.05213961707170117
s_tff1 = 0.033639836997211896
20221230_165912_MH_MN10Model_parameterEstimates_parmset3_cost_294.44.csv
CONC2_init = 0.042454863734382786
CONC3_init = 0.16594963559219386
CONC4_init = 0.2366126557558769
CON

20240522_093119_MH_MN10Model_parameterEstimates_parmset15_cost_294.44.csv
CONC2_init = 0.042442342761781486
CONC3_init = 0.1659041561380048
CONC4_init = 0.23654876184562
CONC5_init = 0.37411611123147054
CONC6_init = 0.8363393232999752
ER_init = 0.12596531425885538
PR_init = 0.7941257143581628
GREB1_init = 0.9134232655360548
TFF1_init = 0.967186893951524
d_er = 0.022274182315735986
d_pr = 0.06560957828316108
d_greb1 = 0.3724517008563879
d_tff1 = 0.0
b_e2er = 0.9999999999955832
d_e2er = 0.24606045389096026
b_e2erpr = 0.02545460547272431
d_e2erpr = 0.8154847406009736
b_e2erprgreb1 = 0.999999998129652
d_e2erprgreb1 = 0.004816963781763075
stim_pr = 67.71015676116919
stim_greb1 = 999.9999999581293
stim_tff1 = 10.480357461444193
s_er = 0.0028057743752607227
s_pr = 0.0521022532228531
s_greb1 = 0.34020604885069955
s_tff1 = 0.0
20240522_093119_MH_MN10Model_parameterEstimates_parmset2_cost_294.44.csv
CONC2_init = 0.042439752623266255
CONC3_init = 0.16589259176778345
CONC4_init = 0.236530974539995